## Tokenization using Byte-Pair Encoding and a Unigram Language Model

Author: Pierre Nugues with help from Marcus Klang

In this assignment, you will create a tokenization program to handle subwords.

In many scripts from Asia, like Chinese, Korean, or Japanese scripts, tokenization cannot rely on white spaces. The byte-pair encoding and the unigram language model are techniques that are now common in machine translation to carry out a tokenization at a subword level. Subword level tokenization shows better multilingual capabilities.

You will follow two papers: 
* Subword Regularization: _Improving Neural Network Translation Models with Multiple Subword Candidates_ by Kudo (2018) (https://arxiv.org/pdf/1804.10959.pdf) and 
* _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020) (https://aclanthology.org/2020.findings-emnlp.414.pdf). 

In addition, you will start from a clear and easy-to-understand description in Google’s Neural Machine Translation System: _Bridging the Gap between Human and Machine Translation_ by Wu et al. (2016). (Do not read them now)
https://arxiv.org/abs/1609.08144

You will use a small corpus make it easier to test and correct your code. Note also that you will use _characters_ and not _bytes_ in this lab as this is simpler to implement. For a complete program, see the link at the end.

**In your report, be sure to answer all the questions. Please reuse the section titles of this notebook so that I can check your answers more easily**

## Preliminaries

As an overall description of the subword tokenizers, read Sections 4 (introduction paragraph) and 4.1. in the paper on translation: _Bridging the Gap between Human and Machine Translation_ by Wu et al. (2016), https://arxiv.org/abs/1609.08144.  

In your report, in a few lines (10 to 15 lines or so) you will:

1. Outline the difference with tokenization as you saw it during the course;
2. Imagine how the tokens will be learned (this will developed in the rest of the lab);
3. Summarize what could be the advantages for Asian languages, unknown words, and translation.

Commenting Sections 4 and 4.1 in your report is **mandatory**. If you are curious, you can read the complete article.

## Understanding SentencePiece
SentencePiece is the combination of BPE and a language model. To be sure you understand how it works, you will first run this code.
These are the first cells from a larger program: https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb
from Google.


In [1]:
%pip install sentencepiece
!wget https: // raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt


Note: you may need to restart the kernel to use updated packages.


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import sentencepiece as spm


Train a model from a corpus. Once trained, read the content of `m.vocab`. Be sure to undestand its content.

In [3]:
spm.SentencePieceTrainer.train(
    '--input=botchan.txt --model_prefix=m --vocab_size=2000')


In [4]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')


True

In [5]:
print(sp.encode('Tokenization using Byte-Pair Encoding'))


[386, 315, 27, 52, 1997, 227, 282, 14, 1058, 237, 22, 717, 1060, 443, 27, 67, 38, 20, 14]


In [6]:
sp.id_to_piece(386)


'▁To'

In [7]:
[sp.id_to_piece(i) for i in range(20)]


['<unk>',
 '<s>',
 '</s>',
 ',',
 '.',
 '▁the',
 '▁I',
 's',
 '▁to',
 '▁a',
 '▁and',
 '▁of',
 '▁',
 'ed',
 'ing',
 '▁in',
 '▁was',
 '▁"',
 '▁it',
 't']

In [8]:
print(sp.encode('Tokenization using Byte-Pair Encoding', out_type=str))


['▁To', 'ke', 'n', 'i', 'z', 'ation', '▁us', 'ing', '▁By', 'te', '-', 'P', 'air', '▁E', 'n', 'c', 'o', 'd', 'ing']


## Design of the BPE Algorithm

The first algorithm to build the subwords from a corpus is a byte-pair encoding (BPE), due to Gage (1994). In the lab, you will first read two sections of more recent articles as they are easier to understand and specifically targeted to natural language processing.

Read these two sections:

1. Section 3.1 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 1 of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, **summarize** (10 to 15 lines or so) with your own words the byte-pair encoding (BPE) algorithm as described by Kudo (2018) and Bostrom and Durrett (2020) (Only BPE and not the unigram language model).

## BPE Programming

You will now program a byte-pair encoding program in Python. You will do it step by step. The first part will be to extract the subwords from a corpus. Note that you will use the characters, not the bytes.

In [9]:
import regex as re
import tqdm as tqdm
import math


First use a small corpus and then, if you have time, test your program on a larger one. Here we take the smallest novel from Selma Lagerlöf in our corpus.

In [10]:
import os
from zipfile import ZipFile
import requests

# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname)
    for fname in
    [
        "bannlyst.txt",
        "gosta.txt",
        "herrgard.txt",
        "jerusalem.txt",
        "kejsaren.txt",
        "marbacka.txt",
        "nils.txt",
        "osynliga.txt",
        "troll.txt"
    ]
]


def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""

    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip donwnloaded.")
        req.close()

        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith(
            "__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)

        print("Done!")


# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")

SELMA_FILES


Selma has been downloaded.


['Selma\\bannlyst.txt',
 'Selma\\gosta.txt',
 'Selma\\herrgard.txt',
 'Selma\\jerusalem.txt',
 'Selma\\kejsaren.txt',
 'Selma\\marbacka.txt',
 'Selma\\nils.txt',
 'Selma\\osynliga.txt',
 'Selma\\troll.txt']

In [11]:
#FILE_PATH = '../../corpus/Selma.txt'
FILE_PATH = 'Selma/herrgard.txt'


Read the corpus and store it in the `corpus` string variable.

In [12]:
with open(FILE_PATH, encoding='utf8') as f:
    corpus = f.read().strip()


Replace all the space sequences in `corpus`, including newlines and tabulations, and normalize them as one space. Use the `\s` class.

In [13]:
# Write your code
corpus = re.sub(r"\s+", " ", corpus)

In [14]:
corpus[:100]


'Selma Lagerlöf En herrgårdssägen Bokutgåva Albert Bonniers förlag, Stockholm 1899. I. Det var en skö'

### BPE

#### Initial Vocabulary

Write the code (one instruction) to split the corpus in a list of characters and store the results in `corpus_l`. This is just a type conversion. Given the input:
<pre><span style="font-size: 12pt;">corpus = 'De senaste fem &aring;ren har cirka 25 000 unga'</span></pre>

Return:
<pre><span style="font-size: 12pt;">corpus_l = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', 'e', ' ', 'f', 'e', 'm', ' ', ...]</span></pre>

In [15]:
# Write your code
corpus_l = list(corpus)


In [16]:
corpus_l[:15]


['S', 'e', 'l', 'm', 'a', ' ', 'L', 'a', 'g', 'e', 'r', 'l', 'ö', 'f', ' ']

Extract the set of characters that will serve as initial subword tokens:

1. Write a statement to extract the set of all the characters from `corpus_l`; 
2. Exclude the space from this set and call the resulting set: `char_set`.

In [17]:
# Write your code
char_set = set(corpus_l)
char_set.remove(" ")

In [18]:
print(sorted(char_set))

['!', ',', '-', '.', '1', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '»', 'Ä', 'Å', 'Ö', 'ä', 'å', 'é', 'ö', '–', '’']


In [19]:
len(char_set)


67

Using code from the previous question, write an `initial_vocabulary()` function taking the the `corpus_l` variable as input and returning the the set of all characters appearing in the corpus (the initial character set), deprived from the white space.

In [20]:
# Write your code here
def initial_vocabulary(corpus_l):
    vocabulary = set(corpus_l)
    vocabulary.remove(" ")
    return sorted(vocabulary)


In [21]:
initial_vocabulary(corpus_l)


['!',
 ',',
 '-',
 '.',
 '1',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '»',
 'Ä',
 'Å',
 'Ö',
 'ä',
 'å',
 'é',
 'ö',
 '–',
 '’']

#### Counting

Write a `pair_count()` function that takes a list of tokens as input, possibly single characters or subword tokens, and that counts the adjacent pairs (bigrams). You will implement these counts as dictionaries: The key will be a pair (tuple) of adjacent symbols and the value, its frequency. Remember that you cannot cross whitespaces, i.e. a pair cannot include a whitespace.

Given the input

`['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', ...]`
count_pairs should return a dictionary: 


`{('D', 'e'): 1, ('s', 'e'): 1, ('e', 'n'): 1, ('n', 'a'): 1, ...}`

In [22]:
# Write your code here
def pair_count(corpus_l):
    vocabulary = initial_vocabulary(corpus_l)
    pairs = {}
    for i in range(len(corpus_l) - 1):
        token1 = corpus_l[i]
        token2 = corpus_l[i+1]
        if token1 in vocabulary and token2 in vocabulary:
            pair = (token1, token2)
            if pair not in pairs:
                pairs[pair] = 1
            else: 
                pairs[pair] += 1

    return pairs


In [23]:
pairs = pair_count(corpus_l)

In [24]:
pairs


{('S', 'e'): 7,
 ('e', 'l'): 511,
 ('l', 'm'): 22,
 ('m', 'a'): 424,
 ('L', 'a'): 8,
 ('a', 'g'): 477,
 ('g', 'e'): 646,
 ('e', 'r'): 1281,
 ('r', 'l'): 213,
 ('l', 'ö'): 70,
 ('ö', 'f'): 217,
 ('E', 'n'): 21,
 ('h', 'e'): 773,
 ('r', 'r'): 177,
 ('r', 'g'): 140,
 ('g', 'å'): 327,
 ('å', 'r'): 295,
 ('r', 'd'): 413,
 ('d', 's'): 83,
 ('s', 's'): 218,
 ('s', 'ä'): 205,
 ('ä', 'g'): 143,
 ('e', 'n'): 3439,
 ('B', 'o'): 2,
 ('o', 'k'): 54,
 ('k', 'u'): 562,
 ('u', 't'): 295,
 ('t', 'g'): 43,
 ('å', 'v'): 2,
 ('v', 'a'): 1426,
 ('A', 'l'): 24,
 ('l', 'b'): 44,
 ('b', 'e'): 314,
 ('r', 't'): 381,
 ('o', 'n'): 1600,
 ('n', 'n'): 1070,
 ('n', 'i'): 231,
 ('i', 'e'): 22,
 ('r', 's'): 258,
 ('f', 'ö'): 980,
 ('ö', 'r'): 1317,
 ('l', 'a'): 873,
 ('g', ','): 192,
 ('S', 't'): 88,
 ('t', 'o'): 351,
 ('o', 'c'): 1223,
 ('c', 'k'): 794,
 ('k', 'h'): 23,
 ('h', 'o'): 1098,
 ('o', 'l'): 211,
 ('1', '8'): 1,
 ('8', '9'): 1,
 ('9', '9'): 1,
 ('9', '.'): 1,
 ('I', '.'): 6,
 ('D', 'e'): 429,
 ('e', 't'): 

Determine the most frequent pair

In [25]:
# write your code
most_freq_pair = max(pairs, key=lambda pair: pairs[pair])

In [26]:
most_freq_pair
pairs[most_freq_pair]


4016

In [27]:
''.join(most_freq_pair)


'de'

#### The First Iteration

We store the initial symbols in a `vocabulary` variable

In [28]:
vocabulary = initial_vocabulary(corpus_l)


In [29]:
len(vocabulary)


67

Add your most frequent pair to the vocabulary after one iteration as well as the merge operations: `merge_ops`. `merge_ops` will contain the merge operations in the order you created them.

In [30]:
merge_ops = []


In [31]:
# write your code here
most_freq_pair = max(pairs, key=lambda pair: pairs.get(pair))
merge_ops.append(most_freq_pair)
vocabulary.append(''.join(most_freq_pair))


In [32]:
len(vocabulary)


68

In [33]:
merge_ops


[('d', 'e')]

#### Incremental Construction
We will now incrementally build the vocabulary.

Create a `merge_bigrams()` function that takes a list of tokens, `corpus_l`, and a pair of subword tokens `(token_r, token_l)` as input and merges adjacent sequences token_r, token_l into a new token, `token_new`, replacing the sequence `token_r, token_l` in `corpus_l`. Your function will return a new list. 

Given the input 

`corpus_l = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', ...]`

`merge_bigrams(corpus_l, ('e', 'n'))` should return where all the seuquences of 'e' and 'n' have been merged:

`['D', 'e', ' ', 's', 'en', 'a', 's', 't', ...]`

And reapplying `merge_bigrams(corpus_l, ('s', 'en'))` to this corpus should return

`['D', 'e', ' ', 'sen', 'a', 's', 't', ...]`

You will apply a greedy algorithm. Given the pair ('a', 'a') and the list ['a', 'a', 'a'], the result will be: ['aa', 'a']

In [34]:
# Write your code here
def merge_bigrams(corpus_l, pair):
    separator = "<SEPARATOR>"
    corpus_l_copy = corpus_l.copy()
    corpus_l_copy.append(separator)
    corpus_l_copy.insert(0, separator)
    new_corpus_l = re.sub(separator + pair[0] + separator + pair[1] + separator, separator + ''.join(pair) + separator, separator.join(corpus_l_copy))
    return new_corpus_l.split(separator)[2:-2]


In [35]:
corpus_test = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', 'e', 'e', 'n']
result = merge_bigrams(corpus_test, ('e', 'n'))
print(result)
print(merge_bigrams(result, ('s', 'en')))



['D', 'e', ' ', 's', 'en', 'a', 's', 't', 'e', 'en']
['D', 'e', ' ', 'sen', 'a', 's', 't', 'e', 'en']


In [36]:
print(merge_bigrams(merge_bigrams(corpus_test, ('e', 'n')), ('s', 'en')))


['D', 'e', ' ', 'sen', 'a', 's', 't', 'e', 'en']


In [37]:
corpus_test = ['a', 'a', 'a']
merge_bigrams(corpus_test, ('a', 'a'))

['aa', 'a']

#### Byte Pair Encoding (BPE): Building the Vocabulary

Write now a `BPE()` function following Algorithm 1 in _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020). 

Your function will take `corpus_l` and the vocabulary size `k` as input. This size `k` will correspond to the count of new subwords added to the initial list of symbols. With your initial corpus, you should have 67 found symbols. With `k = 10`, you will add 10 subwords to this initial list. Note that Bostrom and Durrett (2020) define their $k_\text{Bostrom and Durrett}$ as `k + initial vocabulary`. 

Return the vocabulary of subword tokens in the form of a list, the initial vocabulary and the subwords you will create, as well as the merge operations in the order you created them

You will start from the initial vocabulary and `k` will be the number of symbols you add to this vocabulary.

In [38]:
def BPE(corpus_l, k):
    vocabulary = initial_vocabulary(corpus_l)
    merge_ops = []
    for i in range(k):
        pairs = pair_count(corpus_l)
        most_freq_pair = max(pairs, key=lambda pair: pairs.get(pair))
        new_vocab = ''.join(most_freq_pair)
        vocabulary.append(new_vocab)
        merge_ops.append(most_freq_pair)
        corpus_l = merge_bigrams(corpus_l, most_freq_pair)
        
    return vocabulary, merge_ops


In [39]:
vocabulary, merge_ops = BPE(corpus_l, 50)

In [40]:
print(' '.join(vocabulary[-50:]))
print(len(vocabulary), len(merge_ops))
print(merge_ops)

de en an tt ar st om on ll ör att ch ade ig er ng och var hon et för sk är ck han or na det ne så än in ej un ill den som fv på ed ag li enne henne id ra hade all ing ta
117 50
[('d', 'e'), ('e', 'n'), ('a', 'n'), ('t', 't'), ('a', 'r'), ('s', 't'), ('o', 'm'), ('o', 'n'), ('l', 'l'), ('ö', 'r'), ('a', 'tt'), ('c', 'h'), ('a', 'de'), ('i', 'g'), ('e', 'r'), ('n', 'g'), ('o', 'ch'), ('v', 'ar'), ('h', 'on'), ('e', 't'), ('f', 'ör'), ('s', 'k'), ('ä', 'r'), ('c', 'k'), ('h', 'an'), ('o', 'r'), ('n', 'a'), ('de', 't'), ('n', 'e'), ('s', 'å'), ('ä', 'n'), ('i', 'n'), ('e', 'j'), ('u', 'n'), ('i', 'll'), ('de', 'n'), ('s', 'om'), ('f', 'v'), ('p', 'å'), ('e', 'd'), ('a', 'g'), ('l', 'i'), ('en', 'ne'), ('h', 'enne'), ('i', 'd'), ('r', 'a'), ('h', 'ade'), ('a', 'll'), ('i', 'ng'), ('t', 'a')]


We build a vocabulary of 50 subwords in addition to our initial set of symbols

In [41]:
len(vocabulary), len(merge_ops)


(117, 50)

In [42]:
print(merge_ops)

[('d', 'e'), ('e', 'n'), ('a', 'n'), ('t', 't'), ('a', 'r'), ('s', 't'), ('o', 'm'), ('o', 'n'), ('l', 'l'), ('ö', 'r'), ('a', 'tt'), ('c', 'h'), ('a', 'de'), ('i', 'g'), ('e', 'r'), ('n', 'g'), ('o', 'ch'), ('v', 'ar'), ('h', 'on'), ('e', 't'), ('f', 'ör'), ('s', 'k'), ('ä', 'r'), ('c', 'k'), ('h', 'an'), ('o', 'r'), ('n', 'a'), ('de', 't'), ('n', 'e'), ('s', 'å'), ('ä', 'n'), ('i', 'n'), ('e', 'j'), ('u', 'n'), ('i', 'll'), ('de', 'n'), ('s', 'om'), ('f', 'v'), ('p', 'å'), ('e', 'd'), ('a', 'g'), ('l', 'i'), ('en', 'ne'), ('h', 'enne'), ('i', 'd'), ('r', 'a'), ('h', 'ade'), ('a', 'll'), ('i', 'ng'), ('t', 'a')]


In [43]:
print(vocabulary)


['!', ',', '-', '.', '1', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '»', 'Ä', 'Å', 'Ö', 'ä', 'å', 'é', 'ö', '–', '’', 'de', 'en', 'an', 'tt', 'ar', 'st', 'om', 'on', 'll', 'ör', 'att', 'ch', 'ade', 'ig', 'er', 'ng', 'och', 'var', 'hon', 'et', 'för', 'sk', 'är', 'ck', 'han', 'or', 'na', 'det', 'ne', 'så', 'än', 'in', 'ej', 'un', 'ill', 'den', 'som', 'fv', 'på', 'ed', 'ag', 'li', 'enne', 'henne', 'id', 'ra', 'hade', 'all', 'ing', 'ta']


In [44]:
print(merge_ops)


[('d', 'e'), ('e', 'n'), ('a', 'n'), ('t', 't'), ('a', 'r'), ('s', 't'), ('o', 'm'), ('o', 'n'), ('l', 'l'), ('ö', 'r'), ('a', 'tt'), ('c', 'h'), ('a', 'de'), ('i', 'g'), ('e', 'r'), ('n', 'g'), ('o', 'ch'), ('v', 'ar'), ('h', 'on'), ('e', 't'), ('f', 'ör'), ('s', 'k'), ('ä', 'r'), ('c', 'k'), ('h', 'an'), ('o', 'r'), ('n', 'a'), ('de', 't'), ('n', 'e'), ('s', 'å'), ('ä', 'n'), ('i', 'n'), ('e', 'j'), ('u', 'n'), ('i', 'll'), ('de', 'n'), ('s', 'om'), ('f', 'v'), ('p', 'å'), ('e', 'd'), ('a', 'g'), ('l', 'i'), ('en', 'ne'), ('h', 'enne'), ('i', 'd'), ('r', 'a'), ('h', 'ade'), ('a', 'll'), ('i', 'ng'), ('t', 'a')]


#### BPE Tokenizer

You will now use the vocabulary you obtained to tokenize a text stored in the corpus string.

You will apply the merge operations in the same order you created them. You will call this function `tokenize_bpe()` that will take two inputs: `corpus` and `merge_ops`, and that will return the tokenized text in the form of a list.

    def tokenize_bpe(corpus, merge_ops):

      ...

      return tokens

This function is easy. Just reuse the `merge_bigrams()` function and a loop

In [45]:
# Write your code here
def tokenize_bpe(corpus, merge_ops):
    corpus_l = list(corpus)
    for merge_pair in merge_ops:
        corpus_l = merge_bigrams(corpus_l, merge_pair)
    return corpus_l


In [46]:
print(tokenize_bpe(corpus, merge_ops)[:200])


['S', 'e', 'l', 'm', 'a', ' ', 'L', 'ag', 'er', 'l', 'ö', 'f', ' ', 'E', 'n', ' ', 'h', 'er', 'r', 'g', 'å', 'r', 'd', 's', 's', 'ä', 'g', 'en', ' ', 'B', 'o', 'k', 'u', 't', 'g', 'å', 'v', 'a', ' ', 'A', 'l', 'b', 'er', 't', ' ', 'B', 'on', 'n', 'i', 'er', 's', ' ', 'för', 'l', 'ag', ',', ' ', 'S', 't', 'o', 'ck', 'h', 'o', 'l', 'm', ' ', '1', '8', '9', '9', '.', ' ', 'I', '.', ' ', 'D', 'et', ' ', 'var', ' ', 'en', ' ', 'sk', 'ö', 'n', ' ', 'h', 'ö', 'st', 'd', 'ag', ' ', 'h', 'ä', 'ne', 'm', 'o', 't', ' ', 's', 'l', 'u', 't', 'et', ' ', 'a', 'f', ' ', 't', 'r', 'e', 'tt', 'i', 'o', 'ta', 'l', 'et', '.', ' ', 'P', 'å', ' ', 'den', ' ', 't', 'i', 'den', ' ', 'f', 'an', 'n', 's', ' ', 'i', ' ', 'U', 'p', 's', 'a', 'l', 'a', ' ', 'e', 'tt', ' ', 'h', 'ö', 'g', 't', ',', ' ', 'g', 'u', 'l', 't', ' ', 't', 'v', 'å', 'v', 'å', 'n', 'ing', 's', 'h', 'u', 's', ',', ' ', 'som', ' ', 'st', 'o', 'd', ' ', 'un', 'de', 'r', 'l', 'ig', 't', ' ', 'en', 's', 'a', 'm', 't', ' ', 'på', ' ', 'en', ' ',

In [47]:
print(len(merge_ops))

50


## Unigram Language Model

You are now done with BPE and you can now consider the unigram language model.

Read these two sections:

1. Section 3.2 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 2 and the related text of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, **summarize** (10 to 15 lines or so) with your own words the tokenization with a unigram language model as described by by Kudo (2018) and Bostrom and Durrett (2020). You will notably consider two aspects:
1. How to obtain the subword vocabulary;
2. How to tokenize a text.

In your report, given what you have done on the byte-pair encoding, how would you build the “reasonably big seed vocabulary” needed for the unigram language model?

### Unigram Probabilities

Starting from the “reasonably big seed vocabulary”, you will now fit a unigram language model. You will start with a vocabulary of 50 subwords in addition to the character set and reduce it to 49, i.e. you will find one subword to discard.

Kudo (2018) proposes the expectation-maximization algorithm. We will ignore this step. Instead, in this lab, you will approximate the language model with the BPE algorithm.

Write a `unigram_lm()` function that takes a corpus string and a vocabulary of subword tokens as input and returns a dictionary, where the keys are the subwords and each key value, the key relative frequency:

    def unigram_lm(corpus, vocabulary):

       ...

      return unigram_probs
Your function will:

1. Tokenize your corpus with BPE (you can reuse the `tokenize_bpe()` function);
2. Estimate the probability of each word (simply count the occurrences of the subwords and divide them by the length of the tokenized corpus);
3. Return this model as a dictionary.

In [48]:
print(merge_ops)
print(len(vocabulary))

[('d', 'e'), ('e', 'n'), ('a', 'n'), ('t', 't'), ('a', 'r'), ('s', 't'), ('o', 'm'), ('o', 'n'), ('l', 'l'), ('ö', 'r'), ('a', 'tt'), ('c', 'h'), ('a', 'de'), ('i', 'g'), ('e', 'r'), ('n', 'g'), ('o', 'ch'), ('v', 'ar'), ('h', 'on'), ('e', 't'), ('f', 'ör'), ('s', 'k'), ('ä', 'r'), ('c', 'k'), ('h', 'an'), ('o', 'r'), ('n', 'a'), ('de', 't'), ('n', 'e'), ('s', 'å'), ('ä', 'n'), ('i', 'n'), ('e', 'j'), ('u', 'n'), ('i', 'll'), ('de', 'n'), ('s', 'om'), ('f', 'v'), ('p', 'å'), ('e', 'd'), ('a', 'g'), ('l', 'i'), ('en', 'ne'), ('h', 'enne'), ('i', 'd'), ('r', 'a'), ('h', 'ade'), ('a', 'll'), ('i', 'ng'), ('t', 'a')]
117


In [60]:
tokenized_corpus = tokenize_bpe(corpus, merge_ops)
tokenized_corpus[:15]

['S', 'e', 'l', 'm', 'a', ' ', 'L', 'ag', 'er', 'l', 'ö', 'f', ' ', 'E', 'n']

In [61]:
# Write your code here
def unigram_lm(tokenized_corpus):
    unigram_probs = {}
    count = 0
    for token in tokenized_corpus:
        if token == " ":
            continue
        if token not in unigram_probs:
            unigram_probs[token] = 1
            count += 1
        else:
            unigram_probs[token] += 1 
            count += 1
    for unigram in unigram_probs:
        unigram_probs[unigram] /= count
    return unigram_probs


In [62]:
unigram_probs = unigram_lm(tokenized_corpus)
unigram_probs


{'S': 0.001719180721463082,
 'e': 0.030579680074300224,
 'l': 0.032901562083172776,
 'm': 0.02879133691001966,
 'a': 0.040548952188991315,
 'L': 0.00037545326100917883,
 'ag': 0.004712926460562587,
 'er': 0.010828862475422632,
 'ö': 0.010068075604430349,
 'f': 0.02101550226754009,
 'E': 0.0002667694222959955,
 'n': 0.017606781871535703,
 'h': 0.01574927626443766,
 'r': 0.030955133335309405,
 'g': 0.024681111736866546,
 'å': 0.02122298959599253,
 'd': 0.029453320291272688,
 's': 0.03798994180474454,
 'ä': 0.015255258815741373,
 'en': 0.02388080346997856,
 'B': 0.0008101886158619122,
 'o': 0.015808558358281214,
 'k': 0.02448350475738803,
 'u': 0.02201341751390659,
 't': 0.044254083054213474,
 'v': 0.014603155783462272,
 'A': 0.0006224619853573228,
 'b': 0.015077412534210709,
 'on': 0.0057108417069290885,
 'i': 0.020679570402426614,
 'för': 0.00906028000908992,
 ',': 0.027091916886504433,
 'ck': 0.007844997085297054,
 '1': 9.88034897392576e-06,
 '8': 9.88034897392576e-06,
 '9': 1.97606979

In [63]:
len(unigram_probs)


117

### Unigram Tokenization

You will now apply your unigram language model to tokenize a character sequence that does not include spaces, typically a single word in the Latin or Greek scripts or a sequence of words in Asian scripts, like Chinese or Korean.

Write a `tokenize_lm()` function that takes a character sequence, `char_seq`, and a dictionary of unigram probabilities, `unigram_probs`,  as input and returns the subword tokens and the segmentation probability, (prob,tokens). You will only return the token list with the highest probability.

    def tokenize_lm(char_seq, unigram_probs):

      ...

      return max(candidates)

As an example, applying 

`tokenize_lm('senare', unigram_probs)`
results in

`(2.0899522820189735e-07, ['s', 'en', 'ar', 'e'])`

Your function will cache (memoize) the results to speed up the computation. It will be similar to that of Norvig's in the notebook: How to Do Things with Words.ipynb. You can reuse it.
Python has a built-in memoization function that you can use: @functools.lru_cache(maxsize=2**10). You can also use the newer @functools.cache() function if you have Python 3.9 or higher. See here: https://docs.python.org/3/library/functools.html.

In [198]:
def tokenize_lm(char_seq, unigram_probs):
    def memo(f):
        "Memoize function f, whose args must all be hashable."
        cache = {}
        def fmemo(*args):
            if args not in cache:
                cache[args] = f(*args)
            return cache[args]
        fmemo.cache = cache
        return fmemo

    def splits(text, start=0, L=20):
        "Return a list of all (first, rest) pairs; start <= len(first) <= L."
        return [(text[:i], text[i:]) 
                for i in range(start, min(len(text), L)+1)]

    def p_tokens(tokens: list):
        log_probability = 1
        for token in tokens:
            log_probability += math.log10(unigram_probs[token]) ## Adding probabilities in logspace
        return math.pow(10, log_probability)

    @memo
    def segment(text):
        if not text: 
            return ([], 0)
        else:
            candidates = []
            for (first, rest) in splits(text, 1):
                if first in unigram_probs:
                    rest_segment = segment(rest)[0]
                    candidate = [first] + rest_segment
                    candidates.append(candidate)
            
            # for candidate in candidates: # (For testing)
            #     print(f"Prob = {str(p_tokens(candidate)):<25s} {str(candidate)}")
            # print()
            best_candidate = max(candidates, key=lambda candidate: p_tokens(candidate))
            probability = p_tokens(best_candidate)
            return (best_candidate, probability)
    
    return segment(char_seq)

In [196]:
tokenize_lm("senare", unigram_probs)

Prob = 0.3057968007430022        ['e']

Prob = 0.009465980740510676      ['r', 'e']

Prob = 0.00038383560046888025    ['a', 'r', 'e']
Prob = 0.0028461391182767522     ['ar', 'e']

Prob = 5.0111350631543706e-05    ['n', 'ar', 'e']
Prob = 6.500139920221048e-05     ['na', 'r', 'e']

Prob = 1.9877219919854672e-06    ['e', 'na', 'r', 'e']
Prob = 6.796808893178519e-05     ['en', 'ar', 'e']

Prob = 2.582103743098219e-06     ['s', 'en', 'ar', 'e']



(['s', 'en', 'ar', 'e'], 2.582103743098219e-06)

In [199]:
tokenize_lm("honomochhenne", unigram_probs)

(['hon', 'om', 'och', 'henne'], 5.0787114286979256e-08)

### Text Tokenization with Unigrams

The previous function applies to a sequence without spaces. You will now apply it to your corpus. Write a `tokenize_text_lm()` function that takes the whole `corpus` string as input and the unigram probabilities `unigram_probs` and return the corpus probability and the tokenized subwords. 

This function is just an application of the functions you just wrote, where you will:
1. `str.split()` the string by whitespaces
2. Break the tokens into subtokens using `tokenize_lm()`. This function will return the probabilities of the resulting sequences;
3. Sum the logarithm of these probabilities. Use log10 to check your output with the numbers in the notebook. 

It is very significant that you use the logarithm of the probabilities and the sum. If you multiply the probabilities, you will get an underflow.

In [203]:
# Write your code
def tokenize_text_lm(corpus, unigram_probs):
    corpus_list = corpus.split(" ")
    corpus_prob = 0.0
    tokenized_corpus = []
    for word in corpus_list:
        result = tokenize_lm(word, unigram_probs)
        tokenized_corpus.append(result[0])
        corpus_prob += math.log10(result[1])
    return corpus_prob, tokenized_corpus


In [204]:
init_loglikelihood, tokens = tokenize_text_lm(corpus, unigram_probs)


In [205]:
init_loglikelihood, tokens[:10]


(-149150.9415604095,
 [['S', 'e', 'l', 'm', 'a'],
  ['L', 'ag', 'er', 'l', 'ö', 'f'],
  ['E', 'n'],
  ['h', 'er', 'r', 'g', 'å', 'r', 'd', 's', 's', 'ä', 'g', 'en'],
  ['B', 'o', 'k', 'u', 't', 'g', 'å', 'v', 'a'],
  ['A', 'l', 'b', 'er', 't'],
  ['B', 'on', 'n', 'i', 'er', 's'],
  ['för', 'l', 'ag', ','],
  ['S', 't', 'o', 'ck', 'h', 'o', 'l', 'm'],
  ['1', '8', '9', '9', '.']])

### Vocabulary Selection

You will now implement the final loop, where you will, at each iteration:
1. Select one subword and remove it from your vocabulary.
2. Estimate the probabilities of the subwords of the reduced vocabulary using `unigram_lm()`
2. Compute the resulting log-likelihood of the corpus without this word. You will use `tokenize_text_lm()` this time
3. Compute the loss, i.e. the log-likelihood reduction when the subword is removed from the current vocabulary

You will always keep the single characters in your vocabulary to avoid unknown words.

Store the pairs, (log-likelihood, removed_subword) in a list `logloss_word` and rank them by likelihood value.

In [207]:
# To have the same results
vocabulary_sorted = sorted(vocabulary, key=lambda w: -unigram_probs[w])
vocabulary_sorted[:10]


['t', 'a', 's', 'l', 'r', 'e', 'd', 'm', ',', 'g']

In [218]:
logloss_word = []
for i, word in enumerate(tqdm.tqdm(vocabulary_sorted)):
    if len(word) == 1:
        continue
    new_unigram_probs = unigram_probs.copy()
    new_unigram_probs.pop(word)
    log_likelyhood, _ = tokenize_text_lm(corpus, new_unigram_probs)
    logloss = log_likelyhood - init_loglikelihood
    logloss_word.append((logloss, word))




  0%|          | 0/117 [00:00<?, ?it/s]

100%|██████████| 117/117 [00:58<00:00,  2.00it/s]


In [219]:
sorted(logloss_word, reverse=True)


[(-6.3704471531673335, 'enne'),
 (-24.146723820391344, 'ne'),
 (-158.9616545587196, 'ta'),
 (-243.61502718477277, 'ra'),
 (-288.20943998580333, 'tt'),
 (-292.89654817263363, 'ag'),
 (-372.3527316812542, 'ed'),
 (-375.25266996462597, 'li'),
 (-383.62161302164895, 'id'),
 (-391.89705506624887, 'ör'),
 (-436.3279362382309, 'll'),
 (-457.85251635246095, 'ch'),
 (-463.2616487822961, 'den'),
 (-491.186149011628, 'all'),
 (-513.3743659523316, 'så'),
 (-536.0307046508242, 'ing'),
 (-548.7078506960825, 'ng'),
 (-609.4751344573742, 'na'),
 (-610.1615978183981, 'som'),
 (-611.2206381156575, 'hade'),
 (-646.7434951196774, 'un'),
 (-659.4052671095124, 'det'),
 (-666.3496203502582, 'fv'),
 (-668.8856950351037, 'in'),
 (-676.0680633586016, 'ar'),
 (-680.5798081529501, 'på'),
 (-740.5383088440867, 'on'),
 (-752.8127610596712, 'ej'),
 (-764.2592574419104, 'ade'),
 (-791.5103152134106, 'sk'),
 (-797.7972857517598, 'et'),
 (-805.2193228699034, 'än'),
 (-846.8185025806306, 'or'),
 (-879.0627905327128, 'il

You will reduce now your vocabulary by one token: `out_candidate`. Write the piece of code to determine it.

In [220]:
# Write your code here
out_candidate = max(logloss_word)


In [222]:
out_word = out_candidate[1]


'enne'

## The tokenization without the subword

In [229]:
new_loglikelihood, tokens = tokenize_text_lm(corpus, new_unigram_probs)


In [231]:
init_loglikelihood

-149150.9415604095

In [232]:
new_loglikelihood


-149157.31200756266

In [233]:
tokens[:10]


[['S', 'e', 'l', 'm', 'a'],
 ['L', 'ag', 'er', 'l', 'ö', 'f'],
 ['E', 'n'],
 ['h', 'er', 'r', 'g', 'å', 'r', 'd', 's', 's', 'ä', 'g', 'en'],
 ['B', 'o', 'k', 'u', 't', 'g', 'å', 'v', 'a'],
 ['A', 'l', 'b', 'er', 't'],
 ['B', 'on', 'n', 'i', 'er', 's'],
 ['för', 'l', 'ag', ','],
 ['S', 't', 'o', 'ck', 'h', 'o', 'l', 'm'],
 ['1', '8', '9', '9', '.']]

If you are interested, you can improve this program and test it on larger corpora. You can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will write a report where you will:
1. Describe the background as well as the algorithms you used. For this, summarize the articles as described in the notebook:
   * Preliminaries: subword tokenizers
   * Design of the BPE Algorithm
   * Unigram Language Model
2. Describe your program as well as your results

The whole report should be of 2 to 3 pages.

Submit your report as well as your **notebook** (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 29, 2023.

## Curious?

If you are interested, you can read the sentencepiece implementation by the author:
https://github.com/google/sentencepiece/tree/master

Concerning BPE, you can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py